## Read model

In [1]:
import encoder

model = encoder.parse_model('models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec')
    
print(len(model))

384765


In [7]:
import pickle

texts = encoder.load_file('data/train_texts_cleaned_short.pickle')

durty_text = texts[1]
print(durty_text)
print(len(durty_text.split()))

cleaned_text = encoder.clean_text(durty_text, model)
print(cleaned_text)
print(len(cleaned_text.split()))

досье коломбие реалить роман этёрн коломбие досье ditions l’arganier егоров перевод палимпсест «издательство «этерна» оформление весь начаться июльский утром быть поехать отпуск свой тогдашний подруга один брюнетка грозный характер отец согласиться дать свой машина совсем новенький «рено» который удовольствие пролетать километр национальный автострада вандей департамент запад франция побережье атлантический океан здесь далее прима веда серия ехать слегка превышать скорость рисковать этот прямая линия зелёный свет светофор гарантировать полный безопасность момент когда приближаться светофор наперерез выскочить какой «фольксваген» резко вывернуть руль избежать столкновение получиться потерять управление «рено» вылететь шоссе пробить ограждение скатиться крутой откос замереть знать всего несколько метр могучий стекло разлететься мелкий осколок поранить лицо софи выбраться груда бесформенный железо дорога собраться зевака успеть вызвать полиция пожарный посередине перекрёсток неподвижно ст